In [66]:
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
import numpy as np
import json
import requests
import pandas as pd

In [131]:
QUERY_NUMBER = "q5"
QUERY_PATH = '../queries/' + QUERY_NUMBER + '/' + QUERY_NUMBER

BASE_QUERY_URL = "http://localhost:8983/solr/games/select?fl=*%2C%5Bchild%5D&fq=%7B!child%20of%3D%22*%3A*%20-_nest_path_%3A*%22%7Dsummary%3A(unique%20OR%20characters%20OR%20%22unique%20story%22)%20OR%20wikipedia%3A(unique%20OR%20characters%20OR%20%22unique%20story%22)&indent=true&q.op=OR&q=review%3A(unique%20OR%20characters%20OR%20%22unique%20story%22)&rows=1000&useParams=&wt=json"
BOOSTED_QUERY_URL = "http://localhost:8983/solr/games/select?bq=review%3A%22unique%20character%22%5E4%20OR%20review%3A%22unique%20story%22&defType=edismax&fl=*%2C%5Bchild%5D&fq=%7B!child%20of%3D%22*%3A*%20-_nest_path_%3A*%22%7Dsummary%3A(unique%20OR%20characters%20OR%20%22unique%20story%22)%20OR%20wikipedia%3A(unique%20OR%20characters%20OR%20%22unique%20story%22)&indent=true&ps2=3&q.op=OR&q=review%3A(unique%20OR%20characters%20OR%20%22unique%20story%22)&rows=1000&useParams=&wt=json"

#### Executa a base query e guarda os documentos obtidos no ficheiro baseRank.txt

In [132]:
#Query retorna reviews
base_results = requests.get(BASE_QUERY_URL).json()['response']['docs']
base_gameids = []

for index, doc in enumerate(base_results):
    if len(base_gameids) == 20:
        break
    
    if doc['id'] not in base_gameids:
        base_gameids.append(doc['id'].split('/')[0])

id_string = 'id%3A(' + '%20OR%20'.join(map(str, base_gameids)) + ')&useParams=&wt=json'
BASE_QUERY_URL = "http://localhost:8983/solr/games/select?fl=*%2C%5Bchild%5D&indent=true&q.op=OR&q=" + id_string

print(BASE_QUERY_URL)



http://localhost:8983/solr/games/select?fl=*%2C%5Bchild%5D&indent=true&q.op=OR&q=id%3A(5fc178c2-5fa3-4f94-bb34-b6d82be0f376%20OR%20a675b60f-ba6b-4ca5-ba2f-9c2bea86e973%20OR%208c64e4f8-fbb1-40c5-a35b-24fe592ec6ce%20OR%20b8fb319e-3203-4e03-b27d-445614d081cf%20OR%2017f6fd20-ee3b-4c8e-a41e-cc656b7e2adb%20OR%2063ce02dc-0ef0-4508-be90-04720a085d71%20OR%2063ce02dc-0ef0-4508-be90-04720a085d71%20OR%20fd80ba3b-d8a8-464d-be79-9a3cf91782bd%20OR%20d9aaecbe-fd6a-4b08-87b5-b61c69e31320%20OR%20a6b5774e-c08a-440c-9c5e-299d7e17481d%20OR%2096a27767-08ee-4ce6-9ee3-d6e884b85162%20OR%2052d1b333-0986-461b-8fbf-0af0219d530e%20OR%206a3a8803-7464-47c7-8cb9-df99596c57e1%20OR%200a269bef-2380-400f-9981-75ae1ab70822%20OR%20c54e87be-e61a-4bb5-ae7a-38d6c3674aa6%20OR%20c54e87be-e61a-4bb5-ae7a-38d6c3674aa6%20OR%205bebb982-f9da-4136-a4f8-71f7fbb46c5b%20OR%20e2f7a27d-c8ae-4c28-b8c8-b23ff11b6f16%20OR%20d3654806-4cc5-4473-b670-5b446cbb9e91%20OR%208e42f6ab-4c2a-49e0-adb1-f6337eed401c)&useParams=&wt=json


In [133]:
#Query retorna reviews
boosted_results = requests.get(BOOSTED_QUERY_URL).json()['response']['docs']
boosted_gameids = []

for index, doc in enumerate(boosted_results):
    if len(boosted_gameids) == 20:
        break
    
    if doc['id'] not in boosted_gameids:
        boosted_gameids.append(doc['id'].split('/')[0])
print(boosted_results)
id_string = 'id%3A(' + '%20OR%20'.join(map(str, boosted_gameids)) + ')&useParams=&wt=json'
BOOSTED_QUERY_URL = "http://localhost:8983/solr/games/select?fl=*%2C%5Bchild%5D&indent=true&q.op=OR&q=" + id_string

print(BOOSTED_QUERY_URL)

[{'platform': 'Game Boy Advance', 'release_date': '2006-08-08T00:00:00Z', 'meta_score': 72, 'user_review': 6.6, 'reviewer': 'Nintendojo', 'review': 'The deep combat, unique character designs and intriguing plot offer something for both the strategy fan and the giant robot enthusiast.', 'id': 'cfdfe3a0-027b-4815-baaa-0122af258cda/reviews#0', '_version_': 1782318514092113920}, {'platform': 'Switch', 'release_date': '2020-10-30T00:00:00Z', 'meta_score': 79, 'user_review': 8.4, 'reviewer': 'Noisy Pixel', 'review': 'Mad Rat Dead is a quality rhythm-based platformer that you may not have been expecting. If you can get past the problematic jump attack, you get to experience a blissful and upbeat soundtrack across an addictive gameplay loop and surprisingly unique story.', 'id': '5fc178c2-5fa3-4f94-bb34-b6d82be0f376/reviews#3', '_version_': 1782318514696093696}, {'platform': 'PlayStation Vita', 'release_date': '2009-07-02T00:00:00Z', 'meta_score': 47, 'user_review': 6.4, 'reviewer': 'Noisy Pix

In [134]:
base_results = requests.get(BASE_QUERY_URL).json()['response']['docs']
base_ranked_doc = [doc['title'] for index, doc in enumerate(base_results)]
df_base = pd.DataFrame(base_ranked_doc, columns=['BASE'], index=None)

print(df_base)

latex_table = df_base.to_latex(index=False)

with open(QUERY_PATH+'_baseRank.txt', 'w') as tf:
    tf.write(latex_table)

                                    BASE
0                          Dead Rising 2
1                    XCOM: Chimera Squad
2                            Apocalipsis
3                            Fae Tactics
4                           Mad Rat Dead
5                 The Punisher: No Mercy
6          Ori and the Will of the Wisps
7               Necrobarista: Final Pour
8                        The Incredibles
9  Tales of Vesperia: Definitive Edition


C:\Users\luisk\AppData\Local\Temp\ipykernel_5048\3385266434.py:7: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df_base.to_latex(index=False)


#### Executa a boosted query e guarda os documentos obtidos no ficheiro boostedRank.txt

In [135]:
boosted_results = requests.get(BOOSTED_QUERY_URL).json()['response']['docs']
boosted_ranked_doc = [doc['title'] for index, doc in enumerate(boosted_results)]

df_boosted = pd.DataFrame(boosted_ranked_doc, columns=['BOOSTED'], index=None)

print(df_boosted)

latex_table = df_boosted.to_latex(index=False)

with open(QUERY_PATH+'_boostedRank.txt', 'w') as tf:
    tf.write(latex_table)


                                   BOOSTED
0                            Dead Rising 2
1                      XCOM: Chimera Squad
2                              Apocalipsis
3                              Fae Tactics
4  Super Robot Taisen: Original Generation
5                             Mad Rat Dead
6                   The Punisher: No Mercy
7            Ori and the Will of the Wisps
8                 Necrobarista: Final Pour
9                          The Incredibles


C:\Users\luisk\AppData\Local\Temp\ipykernel_5048\3021056577.py:8: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df_boosted.to_latex(index=False)


#### Depois construir o ficheiro com todos os documentos relevantes

In [ ]:
relevant = list(map(lambda el: el.strip(), open(QUERY_PATH+'_relevant.txt').readlines()))

print(relevant)

#### Definição das métricas

In [ ]:
metrics = {}
metric = lambda f: metrics.setdefault(f.__name__, f)

@metric
def p10(results, relevant, n=10):
    """Precision at N"""
    return len([doc for doc in results[:n] if doc['id'] in relevant])/n

@metric
def ap(results, relevant):
    """Average Precision"""
    precision_values = []
    relevant_count = 0

    for idx, doc in enumerate(results):
        if doc['id'] in relevant:
            relevant_count += 1
            precision_at_k = relevant_count / (idx + 1)
            precision_values.append(precision_at_k)

    if not precision_values:
        return 0.0

    return sum(precision_values)/len(precision_values)

def calculate_metric(key, results, relevant):
    return metrics[key](results, relevant)

evaluation_metrics = {
    'ap': 'Average Precision',
    'p10': 'Precision at 10 (P@10)'
}

#### Função que cálcula a curva de Precision-Recall

In [ ]:
def evaluate(results, queryType):
    precision_values = [
        len([
            doc 
            for doc in results[:idx]
            if doc['id'] in relevant
        ]) / idx 
        for idx, _ in enumerate(results, start=1)
    ]
    
    recall_values = [
        len([
            doc for doc in results[:idx]
            if doc['id'] in relevant
        ]) / len(relevant)
        for idx, _ in enumerate(results, start=1)
    ]
    
    precision_recall_match = {k: v for k,v in zip(recall_values, precision_values)}
    
    # Extend recall_values to include traditional steps for a better curve (0.1, 0.2 ...)
    recall_values.extend([step for step in np.arange(0.1, 1.1, 0.1) if step not in recall_values])
    recall_values = sorted(set(recall_values))

    # Extend matching dict to include these new intermediate steps
    for idx, step in enumerate(recall_values):
        if step not in precision_recall_match:
            if recall_values[idx-1] in precision_recall_match:
                precision_recall_match[step] = precision_recall_match[recall_values[idx-1]]
            else:
                precision_recall_match[step] = precision_recall_match[recall_values[idx+1]]

    disp = PrecisionRecallDisplay([precision_recall_match.get(r) for r in recall_values], recall_values)
    disp.plot()
    plt.savefig(QUERY_PATH+'_'+queryType+'_pr.pdf')
    

#### Cálculo da curva para a base query

In [ ]:
evaluate(base_results, 'base')

#### Cálculo da curva para a boosted query

In [ ]:
evaluate(boosted_results, 'boosted')

In [ ]:
base_document_ids = [doc['id'] for doc in base_results]
base_relevance_column = ['Y' if doc_id in relevant else 'N' for doc_id in base_document_ids]

boosted_document_ids = [doc['id'] for doc in boosted_results]
boosted_relevance_column = ['Y' if doc_id in relevant else 'N' for doc_id in boosted_document_ids]

ranked_documents = list(zip(range(1, len(base_results) + 1), base_document_ids, base_relevance_column, boosted_document_ids, boosted_relevance_column))

df1 = pd.DataFrame(ranked_documents, columns=[('Rank', ''), ('Base System', 'Game'), ('Base System', 'Relevance'), ('Boosted System', 'Game'), ('Boosted System', 'Relevance')])

# Convert the DataFrame to a LaTeX table without an index
latex_table = df1.to_latex(index=False)

# Write the LaTeX table to a file
with open(QUERY_PATH+'_ranked_documents.tex', 'w') as tf:
    tf.write(latex_table)

# Print the LaTeX table
print(latex_table)